In [1]:
!pip uninstall keras
!pip uninstall tensorflow
!pip uninstall opencv-python

Uninstalling Keras-2.4.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/Keras-2.4.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/docs/*
    /usr/local/lib/python3.7/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.7/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.4.3
Uninstalling tensorflow-2.4.1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.4.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.4.1
Uninstalling opencv-python-4.1.2.30:
  Would r

In [2]:
!pip install keras==2.0.8
!pip install tensorflow==1.14.0
!pip install opencv-python==3.4.2.17

     |████████████████████████████████| 276kB 6.0MB/s 
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.8 which is incompatible.
     |████████████████████████████████| 109.3MB 105kB/s 
     |████████████████████████████████| 3.2MB 36.6MB/s 
     |████████████████████████████████| 491kB 35.8MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
     |████████████████████████████████| 25.0MB 128kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/MyDrive/vgg_promark/Image_Segmentation/Instance_Segmentation

/content/gdrive/MyDrive/vgg_promark/Image_Segmentation/Instance_Segmentation


In [5]:
!pip install mrcnn==0.1

     |████████████████████████████████| 61kB 3.2MB/s 
  Created wheel for mrcnn: filename=mrcnn-0.1-cp37-none-any.whl size=54933 sha256=65331c0a6826fb27505c9ac55ba988725eff0a92529a1072ff5242719f98b249
  Stored in directory: /root/.cache/pip/wheels/bf/2e/96/1b75d3e31ce8db33e7f02e67022ededfacbcce5ccf23ca4104
Successfully built mrcnn


In [6]:
import keras
keras.__version__

Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

'2.0.8'

In [7]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import random
import collections

# Root directory of the project
ROOT_DIR = "/content/gdrive/MyDrive/vgg_promark/Image_Segmentation/Instance_Segmentation"
ROOT_IMAGE_DIR = os.path.abspath("/content/gdrive/MyDrive/vgg_promark/Image_Segmentation/")

# Path to the dataset (note this is a shared images directory)
dataset_path = os.path.join(ROOT_IMAGE_DIR, "train")

models_dir = os.path.join(ROOT_DIR, "models/")

print("base dataset dir:", dataset_path)
print("base modesl dir:", models_dir)

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.model import log

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

base dataset dir: /content/gdrive/MyDrive/vgg_promark/Image_Segmentation/train
base modesl dir: /content/gdrive/MyDrive/vgg_promark/Image_Segmentation/Instance_Segmentation/models/


In [8]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [9]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import det
%matplotlib inline

In [10]:
import imgaug as ia
from imgaug import augmenters as iaa

ia.seed(1)

# http://imgaug.readthedocs.io/en/latest/source/augmenters.html#sequential
seq_of_aug = iaa.Sequential([
    iaa.Crop(percent=(0, 0.1)), # random crops
    
    # horizontally flip 50% of the images
    #iaa.Fliplr(0.5), # Does not make sense for signs

    # Gaussian blur to 50% of the images
    # with random sigma between 0 and 0.5.
    iaa.Sometimes(0.4,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    
    # Strengthen or weaken the contrast in each image.
    iaa.ContrastNormalization((0.75, 1.5)),
    
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    
    # Apply affine transformations to each image.
    # Scale/zoom them from 90% 5o 110%
    # Translate/move them, rotate them
    # Shear them slightly -2 to 2 degrees.
    iaa.Affine(
        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-5, 5),
        shear=(-2, 2)
    )
], random_order=True) # apply augmenters in random order

print("dataset dir:", dataset_path)
print("modesl dir:", models_dir)

dataset dir: /content/gdrive/MyDrive/vgg_promark/Image_Segmentation/train
modesl dir: /content/gdrive/MyDrive/vgg_promark/Image_Segmentation/Instance_Segmentation/models/


In [11]:
import os
import sys
import json
import numpy as np
import skimage.draw
import collections
from collections import defaultdict

# Root directory of the project
ROOT_DIR = "/content/gdrive/MyDrive/vgg_promark/Image_Segmentation/Instance_Segmentation"

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils


In [12]:
class DetConfig(Config):
    """Configuration for training on the dataset.
    Derives from the base Config class and overrides some values.
    """

    def __init__(self, dataset_name, classnames):
        # Give the configuration a recognizable name
        self.dataset_name = dataset_name
        self.NAME = dataset_name

        self.CLASS_NAMES = classnames
        self.ALL_CLASS_NAMES = ['BG'] + self.CLASS_NAMES

        # Number of classes (including background)
        self.NUM_CLASSES = len(self.ALL_CLASS_NAMES)
        self.map_name_to_id = {}
        Config.__init__(self)

    # A GPU with 12GB memory can fit two images.
    IMAGES_PER_GPU = 1

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

    TRAINING_VERBOSE = 1

    TRAIN_BN = False
    #  'relu' or 'leakyrelu'
    ACTIVATION = 'relu'


In [13]:
class DetDataset(utils.Dataset):

    def __init__(self, config):
        self.dataset_name = config.NAME
        self.map_name_to_id = {}
        self.actual_class_names = collections.Counter()
        utils.Dataset.__init__(self)

    def load_by_annotations(self, dataset_dir, annotations_list, class_names):
        """Load a specific set of annotations and from them images.
        dataset_dir: Root directory of the dataset.
        annotations_list: The annotations (and images) to be loaded.
        class_names: List of classes to use.
        """

        # Find the unique classes and track their count
        for a in annotations_list:
            regions = a['regions']
            for a in regions:
                c = a['region_attributes']
                object_name = c['Crimps']
                self.actual_class_names[object_name] += 1

        # Add classes. Use class_names to ensure consistency.
        for i, name in enumerate(class_names):
            # Skip over background if it appears in the class name list
            index = i + 1
            if name != 'BG':
                print('Adding class {:3}:{}'.format(index, name))
                self.add_class(self.dataset_name, index, name)
                self.map_name_to_id[name] = index

        # Add images
        for a in annotations_list:
            # Get the x, y coordinates of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']]

            r_object_name = [r['region_attributes']['Crimps'] for r in a['regions']]

            assert len(polygons) == len(r_object_name)

            # load_mask() needs the image shape.
            image_path = os.path.join(dataset_dir, a['filename'])

            # The annotation file needs to be pre-processed to save the shape of the image.
            # If it isn't it will have to be read in.
            if 'height' in a and 'width in a':
                height = a['height']
                width = a['width']
            else:
                image = skimage.io.imread(image_path)
                height, width = image.shape[:2]

            self.add_image(
                self.dataset_name,
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                r_object_name=r_object_name)

    def load_dataset_images(self, dataset_dir, subset, class_names):
        """Load a subset of the dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        class_names: List of classes to use.
        """

        # Train or validation dataset?
        assert subset in ["train", "val"]

        print(dataset_dir)
        print(subset)

        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        annotations = json.load(open(os.path.join(dataset_dir, "annotations.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        # Find the unique classes and track their count
        for a in annotations:
            for id, region in a['regions'].items():
                object_name = region['region_attributes']['object_name']
                self.actual_class_names[object_name] += 1

        # Add classes.
        for i, name in enumerate(class_names):
            # Skip over background if it occurs in the
            index = i + 1
            if name != 'BG':
                print('Adding class {:3}:{}'.format(index, name))
                self.add_class('wolf', index, name)

        # Add images
        for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']]

            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                self.dataset_name,
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons)

    def load_mask(self, image_id):
        """Generate instance masks for an image.
        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_names: a 1D array of class IDs of the instance masks.
        """

        # If not an object in our dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] not in self.class_names:
            print("warning: source {} not part of our classes, delegating to parent.".format(image_info["source"]))
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        class_ids = []
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            try:
                rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
                mask[rr, cc, i] = 1
                class_id = self.map_name_to_id[info['r_object_name'][i]]
                class_ids.append(class_id)
            except:
                print(image_info)
                raise

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        class_ids = np.array(class_ids, dtype=np.int32)
        return mask.astype(np.bool), class_ids

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == self.dataset_name:
            return info["path"]
        else:
            print("warning: DetDataSet: using parent image_reference for: ", info["source"])
            super(self.__class__, self).image_reference(image_id)


In [14]:
def split_annotations(dataset_dir, config, train_pct=.8, annotation_filename="annotations.json", randomize=True):
    """ divide up an annotation file for training and validation
    dataset_dir: location of images and annotation file.
    config: config object for training.
    train_pct: the split between train and val default is .8
    annotation_filename: name of annotation file.
    randomize: If true (default) shuffle the list of annotations.
    """

    indexes = {}
    for idx, cn in enumerate(config.CLASS_NAMES):
        indexes[cn] = idx

    # Load annotations
    annotations = json.load(open(os.path.join(dataset_dir, annotation_filename)))
    annotations = list(annotations.values())

    # The VIA tool saves images in the JSON even if they don't have any
    # annotations. Skip unannotated images.
    annotations = [a for a in annotations if a['regions']]
    #print(annotations)

    if randomize:
        # Randomize the annotations then divide
        np.random.shuffle(annotations)

    # Find the unique classes and track their count
    uniq_class_names = collections.Counter()
    images_classes = {}
    total_classes = 0
    for a in annotations:
        name = a['filename']
        #b = a['regions']
        rc = np.zeros(len(config.CLASS_NAMES))
        for aa in a['regions']:
            c = aa['region_attributes']
            object_name = c['Crimps']
            uniq_class_names[object_name] += 1
            total_classes += 1
            rc[indexes[object_name]] += 1
        images_classes[name] = rc

    # Calculate the weights for assigning to buckets,
    # the fewer the greater the weight.
    class_weights = np.zeros(len(config.CLASS_NAMES))
    for cn in uniq_class_names:
        class_weights[indexes[cn]] = total_classes / uniq_class_names[cn]

    # Distribute the annotations into buckets by class
    bucket_of_classes = defaultdict(list)
    for a in annotations:
        # Multiply class count by weights to select which bucket.
        t = images_classes[a['filename']] * class_weights
        selected_class = t.argmax()
        bucket_of_classes[config.CLASS_NAMES[selected_class]].append(a)

    train_ann = []
    val_ann = []
    for k, v in bucket_of_classes.items():
        n_for_train = int(len(v)*train_pct)
        train_ann = train_ann + v[:n_for_train]
        val_ann = val_ann + v[n_for_train:]

    def validate_unique(ann, img_files={}):
        for a in ann:
            filename = a['filename']
            if filename in img_files:
                raise RuntimeError(filename+' already exists')
            else:
                img_files[filename] = 1
        return img_files

    img_files = validate_unique(train_ann)
    img_files = validate_unique(val_ann, img_files)
    assert len(train_ann)+len(val_ann) == len(img_files)

    return train_ann, val_ann


In [15]:
def create_datasets(dataset_dir, config, train_pct=.8):
    """ set up the training and validation training set.
    dataset_dir: location of images and annotation file.
    config: config object that includes list of classes being trained for.
    train_pct: the split between train and val default is .8
    """

    train_ann, val_ann = split_annotations(dataset_dir, config, train_pct=train_pct)

    print(annotation_stats(train_ann))
    print(annotation_stats(val_ann))

    train_ds = DetDataset(config)
    train_ds.load_by_annotations(dataset_dir, train_ann, config.CLASS_NAMES)

    val_ds = DetDataset(config)
    val_ds.load_by_annotations(dataset_dir, val_ann, config.CLASS_NAMES)

    assert len(train_ds.image_info) == len(train_ann) and len(val_ds.image_info) == len(val_ann)

    return train_ds, val_ds


def annotation_stats(annotations):
    # Find the unique classes and track their count
    uniq_class_names = collections.Counter()
    for a in annotations:
        for aa in a['regions']:
          c = aa['region_attributes']
          object_name = c['Crimps']
          uniq_class_names[object_name] += 1
    return uniq_class_names

In [16]:
config = DetConfig('train', ['Copper', 'Belmouth'])
dataset_train, dataset_val = create_datasets('/content/gdrive/MyDrive/vgg_promark/Image_Segmentation/train/train', config)

dataset_train.prepare()
dataset_val.prepare()

Counter({'Copper': 2008, 'Belmouth': 879})
Counter({'Copper': 461, 'Belmouth': 205})
Adding class   1:Copper
Adding class   2:Belmouth
Adding class   1:Copper
Adding class   2:Belmouth


In [17]:
print("Training Images: {}\nClasses: {}".format(len(dataset_train.image_ids), dataset_train.class_names))
print("Validation Images: {}\nClasses: {}".format(len(dataset_val.image_ids), dataset_val.class_names))

Training Images: 942
Classes: ['BG', 'Copper', 'Belmouth']
Validation Images: 236
Classes: ['BG', 'Copper', 'Belmouth']


In [18]:
# Setup configuration
config = DetConfig('train', ['Copper', 'Belmouth'])
config.display()


Configurations:
ACTIVATION                     relu
ALL_CLASS_NAMES                ['BG', 'Copper', 'Belmouth']
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
CLASS_NAMES                    ['Copper', 'Belmouth']
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_cl

In [19]:
import keras
keras.__version__

'2.0.8'

In [ ]:
#https://github.com/sabderra/Mask_RCNN/tree/csci_e89_project/csci_e89_project
#https://github.com/theislab/scgen/issues/14

In [20]:
!pip show tensorflow

Name: tensorflow
Version: 1.14.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: gast, six, keras-applications, tensorflow-estimator, numpy, termcolor, wrapt, grpcio, google-pasta, protobuf, tensorboard, keras-preprocessing, astor, absl-py, wheel
Required-by: fancyimpute


In [21]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
model = modellib.MaskRCNN(mode='training', config=config, model_dir=models_dir)

model.keras_model.summary()






Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_image (InputLayer)         (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, None, None, 3) 0           input_image[0][0]                
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, None, None, 64 9472        zero_p

In [22]:
# Load the weights

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(models_dir, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)
else:
    print("using existing ", COCO_MODEL_PATH)
    
# For the coco dataset exclude the last layers because 
# it requires a matching number of classes
print("loading...", end='')
model.load_weights(COCO_MODEL_PATH, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])

print("loaded.")

using existing  /content/gdrive/MyDrive/vgg_promark/Image_Segmentation/Instance_Segmentation/models/mask_rcnn_coco.h5
loading...loaded.


In [23]:
def train(model, dataset_path, epochs=5):
    """Train the model."""
    
    # Create the train and val dataset.
    dataset_train, dataset_val = create_datasets(dataset_path+'/train', config)

    # Prepare them
    dataset_train.prepare()
    dataset_val.prepare()

    # Experiment with training options.
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    history = model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=epochs,
                layers='heads', 
                augmentation=seq_of_aug
                )
    
    return history

In [24]:
import keras
keras.__version__

'2.0.8'

In [25]:
history = train(model, dataset_path, 5)

Counter({'Copper': 1971, 'Belmouth': 861})
Counter({'Copper': 498, 'Belmouth': 223})
Adding class   1:Copper
Adding class   2:Belmouth
Adding class   1:Copper
Adding class   2:Belmouth
Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /content/gdrive/MyDrive/vgg_promark/Image_Segmentation/Instance_Segmentation/models/train20210408T1536/mask_rcnn_train_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistri

/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor




/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1987: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/5


...

ERROR:root:Error processing image {'id': 'bad (63).jpg', 'source': 'train', 'path': '/content/gdrive/MyDrive/vgg_promark/Image_Segmentation/train/train/bad (63).jpg', 'width': 1280, 'height': 720, 'polygons': [{'name': 'polygon', 'all_points_x': [488, 562, 567, 579, 592, 591, 615, 660, 669, 635, 601, 595, 578, 575, 574, 561, 533, 492, 487, 480, 475], 'all_points_y': [76, 70, 54, 34, 35, 60, 68, 77, 203, 215, 221, 202, 202, 220, 229, 234, 235, 232, 214, 202, 192]}, {'name': 'polygon', 'all_points_x': [465, 499, 523, 538, 560, 569, 583, 591, 619, 631, 606, 590, 576, 553, 533, 504, 493, 482, 477, 468, 466], 'all_points_y': [498, 501, 501, 500, 510, 519, 508, 502, 498, 494, 510, 519, 528, 535, 538, 541, 535, 524, 522, 518, 512]}, {'name': 'polygon', 'all_points_x': [453, 438, 429, 431, 436, 444, 462, 486, 506, 522, 532, 552, 559, 574, 585, 602, 621, 626, 639, 655, 673, 691, 713, 717, 725, 730, 728, 707, 687, 619, 608, 596, 588, 584, 563, 518, 488, 484, 469, 450], 'all_points_y': [435, 470,

ERROR:root:Error processing image {'id': 'bad_new (83).jpg', 'source': 'train', 'path': '/content/gdrive/MyDrive/vgg_promark/Image_Segmentation/train/train/bad_new (83).jpg', 'width': 1280, 'height': 720, 'polygons': [{'name': 'polygon', 'all_points_x': [405, 498, 542, 557, 508, 424, 406, 403, 399, 395, 393], 'all_points_y': [79, 81, 83, 217, 229, 234, 228, 202, 180, 166, 157]}, {'name': 'polygon', 'all_points_x': [790, 820, 861, 856, 845, 832, 836, 842, 839, 841, 836, 836, 788, 793, 786, 782], 'all_points_y': [82, 84, 95, 118, 120, 120, 145, 158, 169, 180, 212, 216, 224, 176, 165, 127]}, {'name': 'polygon', 'all_points_x': [809, 809, 780, 760, 754, 757, 762, 758, 758, 758, 760, 754, 748, 743, 760, 768, 787, 793, 788, 781, 785, 785, 787, 789, 789, 789, 790, 790, 797, 801], 'all_points_y': [63, 39, 39, 45, 65, 84, 108, 136, 165, 174, 206, 236, 265, 277, 288, 284, 284, 267, 256, 212, 178, 168, 158, 135, 114, 103, 88, 80, 68, 65]}], 'r_object_name': ['Copper', 'Copper', 'Belmouth']}
Trace

ERROR:root:Error processing image {'id': 'WIN_20210301_16_25_59_Pro.jpg', 'source': 'train', 'path': '/content/gdrive/MyDrive/vgg_promark/Image_Segmentation/train/train/WIN_20210301_16_25_59_Pro.jpg', 'width': 1280, 'height': 720, 'polygons': [{'name': 'polygon', 'all_points_x': [653, 687, 712, 710, 692, 711, 716, 711, 711, 708, 678, 661, 658], 'all_points_y': [350, 344, 345, 387, 413, 444, 462, 483, 496, 502, 504, 502, 501]}], 'r_object_name': ['Belmouth']}
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/mrcnn/model.py", line 1704, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/usr/local/lib/python3.7/dist-packages/mrcnn/model.py", line 1261, in load_image_gt
    hooks=imgaug.HooksImages(activator=hook))
  File "/usr/local/lib/python3.7/dist-packages/imgaug/augmenters/meta.py", line 470, in augment_image
    return self.augment_images([image], hooks=hooks)[0]
  File "/usr/local/lib/python3.7/dist-packages/imgaug/augmenters/meta.py"

ERROR:root:Error processing image {'id': 'WIN_20210309_14_32_19_Pro.jpg', 'source': 'train', 'path': '/content/gdrive/MyDrive/vgg_promark/Image_Segmentation/train/train/WIN_20210309_14_32_19_Pro.jpg', 'width': 1280, 'height': 720, 'polygons': [{'name': 'polygon', 'all_points_x': [428, 357, 349, 347, 359, 339, 349, 340, 338, 346, 356, 349, 359, 360, 359, 354, 361, 372, 396, 415, 427, 429, 422, 410, 417, 423, 429], 'all_points_y': [190, 189, 189, 202, 220, 232, 255, 262, 280, 289, 291, 305, 320, 335, 357, 374, 385, 394, 398, 398, 381, 360, 325, 294, 276, 228, 214]}, {'name': 'polygon', 'all_points_x': [499, 442, 423, 419, 420, 439, 431, 423, 420, 416, 434, 426, 415, 419, 430, 457, 482, 506, 511, 527, 528, 519], 'all_points_y': [145, 139, 139, 155, 172, 184, 216, 254, 288, 303, 348, 397, 412, 423, 430, 422, 411, 404, 318, 249, 182, 151]}, {'name': 'polygon', 'all_points_x': [504, 463, 446, 445, 438, 445, 458, 475, 484, 490, 493, 498, 499, 496, 486, 483, 499], 'all_points_y': [504, 497, 50

StopIteration: ignored

In [ ]:
model_path = os.path.join(MODEL_DIR, "mask_rcnn_Crimps.h5")
model.keras_model.save_weights(model_path)

In [ ]:
import keras
keras.__version__

'2.3.1'